# Deterministic Growth Model Dynamic Program

(This is my version of the example at [Sargent and Stachurski's quant-econ website](http://quant-econ.net/jl/dp_intro.html). Please observe the license file at the root of this repository.)

In this notebook we'll implement the deterministic growth model as a dynamic programming problem. We will assume log utility to get a closed form solution. 
Remember that the problem is defined as
$$ \begin{align}
   V(k) &= \max_{0<k'<f(k)} \ln(f(k) - k') + \beta V(k')\\
     f(k)   & = k^\alpha\\
     k_0   & \text{given} 
     \end{align}
$$

## Representing a function on $\mathbb{R}$ in a computer
     
* The first thing to realise is that we cannot represent $V(k),k\in \mathbb{R}$ on a computer.  However, we can get an arbitrarily good approximation to  $\mathbb{R}$. 
* We will approximate $k$ at a finite number of points $K={k_1,\dots,k_n}$, called a *grid*. 
* In other words, we will compute $V(k)$ above only at the list of points in $K$. 
* There is a slight complication that arises from the $\max$ operator: 
    * Ideally, we would like to choose $c$ out of the **continuous** interval $[0,f(k)]$, and not be restricted to the grid $K$. 
    * In order to achieve this, we must find a way to represent $V(k)$ for values off the grid. 
    * In other words, we will know a list of values $V(k_1),\dots,V(k_n)$, but will most of the time need a value $V(x),x\in (k_i,k_{i+1})$ when we perform the operation $\max_{0<k'<f(k)}$. 
    * We will *linearly interpolate* such a value $x$, which is similar to *connecting the dots*.

## Set some parameter values:

In [ ]:
alpha     = 0.65
beta      = 0.95
grid_max  = 2  # upper bound of capital grid
n         = 150
kgrid     = 1e-6:(grid_max-1e-6)/(n-1):grid_max  # equispaced grid
f(x) = x^alpha  # defines the production function f(k)

Next, because of our log assumption, we know that there is a closed form solution here. It is characterized by 2 constants $c_1,c_2$. We know the true solution to the value function, denoted $V^*$:

In [ ]:
ab        = alpha * beta
c1        = (log(1 - ab) + log(ab) * ab / (1 - ab)) / (1 - beta)
c2        = alpha / (1 - ab)
v_star(k) = c1 .+ c2 .* log(k)	# this defines a function v_star

We will now apply the bellman operator to the functional in the above definition. The operator takes a current guess $V^i$ and returns the next iterate $V^{i+1}$. We define the operator as
$$\begin{align} T(V)(k) =& \max_{0<k'<f(k)} \ln(f(k) - k') + \beta V(k') \\
                V^{i+1}(k) =& \max_{0<k'<f(k)} \ln(f(k) - k') + \beta V^{i}(k') 
                \end
                {align}$$

In [ ]:
# bellman operator function
# that takes a grid and a vector of current function values
# input: * grid: vector of capital at which V is defined
#        * w: vector of values with current function values
# output: * Tw: vector of next iteration function values
function bellman_operator(grid, w)
    
    # 1) we need an object that interpolates the current guess in w
    
    # 2) create a vector to hold the result

    # 3) for all grid points k, do the maximization
        # 4) at each grid point, define an objective function
            # 5) and find the max of that function. 
            # 6) save that in the result vector

    # 7) return the next guess
end

In [ ]:
# we will use some tools from the Grid package to interpolate:
using Grid: CoordInterpGrid, BCnan, InterpLinear
# and to the function "optimize" to the max operation:
using Optim: optimize

# bellman operator function
# that takes a grid and a vector of current function values
# input:  * grid: vector of capital at which V is defined
#         * w: vector of values with current function values
# output: * Tw: vector of next iteration function values
function bellman_operator(grid, w)
 
    # 1) we need an object that interpolates the current guess in w
    Interp = CoordInterpGrid(grid, w, BCnan, InterpLinear)
    
    # 2) create a vector to hold the result
    Tw = zeros(w)

    # 3) for all grid points k, do the maximization
   for (i, k) in enumerate(grid)
        # 4) at each grid point, define an objective function
        objective(c) = - log(c) - beta * Interp[f(k) - c]
        # 5) and find the max of that function. 
        # find max of ojbective between [0,k^alpha]
        res = optimize(objective, 1e-6, f(k)) 
        # 6) save that in the result vector
        Tw[i] = - objective(res.minimum)
    end
    #7) return the next guess
    return Tw
end

We are now ready to start the Value function iteration. We will call `bellman_operator` for a couple of times now and see how the convergence proceeds.

In [ ]:
# value function iteration (VFI) 
# input:  * Integer maxIter: max number of iterations
# output: * matrix Vfuns: each column is an iterate on V
function VFI(grid,V0::Vector,maxIter::Int)
    w = zeros(length(grid),maxIter)
    w[:,1] = V0 # initial condition
    # start iteration
    for i=2:maxIter
        w[:,i] = bellman_operator(grid, w[:,i-1])
    end
    return w
end


In [ ]:
# let's do it!
v0 = 5 .* log(kgrid) .- 25  #initial condition
V = VFI(kgrid,v0,35);

In [ ]:
# setup a plot using the PyPlot package
using PyPlot
function plotVfun(V::Matrix,grid)
    fig, ax = subplots()
    maxIter = size(V,2)
    ax[:set_ylim](-40, -20)
    ax[:set_xlim](minimum(grid), maximum(grid))
    lb = "initial condition"
    jet = ColorMap("jet")[:__call__]
    # plot initial guess
    ax[:plot](grid, V[:,1], color=jet(0), lw=2, alpha=0.6, label=lb)
    # plot other iterates
    for i=2:maxIter
        ax[:plot](grid, V[:,i], color=jet(i/maxIter), lw=2, alpha=0.6)
    end
    # plot true value function
     lb = "true value function"
    # we defined the analytic "v_star" above
    ax[:plot](grid, v_star(grid), "k-", lw=2, alpha=0.8, label=lb)
    ax[:legend](loc="upper left")
    return fig
end

In [ ]:
plotVfun(V,kgrid)

If we run this for longer, we will get closer to the truth:

In [ ]:
v0 = 5 .* log(kgrid) .- 25  #initial condition
V = VFI(kgrid,v0,70);
plotVfun(V,kgrid)

## what about a different initial condition?

In [ ]:
v0 = ones(length(kgrid)) .* -30.0  #a straight line at -30
V = VFI(kgrid,v0,35);
plotVfun(V,kgrid)